In [ ]:
import importlib
from dasilva_invariants import diagnostics, meshes, invariants
importlib.reload(diagnostics)
importlib.reload(meshes)
importlib.reload(invariants)

from datetime import datetime
import numpy as np
import pylab as plt

In [ ]:
def lstar_vs_nlocaltimes_experiment(mesh, max_points=20):
    Lstars = {}
    for i in range(2, max_points):
        r = 7
        th = 0
        mirror_deg = 7.5
        x, y = np.cos(np.deg2rad(th)) * r, np.sin(np.deg2rad(th)) * r
        Lstars[i] = invariants.calculate_LStar(mesh, (x, y, 0), mirror_deg, num_local_times=i)
    return Lstars

In [ ]:
def plot_experiment(Lstars, model_title):
    plt.plot(Lstars.keys(), np.array([res.LStar for res in Lstars.values()]), 'o')
    plt.ylabel('L*')
    plt.xlabel('Number of Local Times')
    plt.title(model_title)
    plt.xticks(list(Lstars.keys()))

# LFM

In [ ]:
mesh = meshes.get_lfm_hdf4_data('../data/20140108_BARREL-LR-small/20140108_BARREL_mhd_2014-01-08T12-48-30Z.hdf')

In [ ]:
Lstars = lstar_vs_nlocaltimes_experiment(mesh)

In [ ]:
plot_experiment(Lstars, 'LFM')

# Dipole

In [ ]:
mesh = meshes.get_dipole_mesh_on_lfm_grid('../data/lfm_data/hdf4/March2015_mhd_2015-03-17T04-30-00Z.hdf')

In [ ]:
Lstars = lstar_vs_nlocaltimes_experiment(mesh)

In [ ]:
plot_experiment(Lstars, 'Dipole')

# T96

In [ ]:
lfm_hdf4_path = '../data/lfm_data/hdf4/March2015_mhd_2015-03-17T04-45-00Z.hdf'
mesh = meshes.get_t96_mesh_on_lfm_grid(
    dynamic_pressure=1,
    Dst=0, 
    By_imf=7 / np.sqrt(2),
    Bz_imf=7 / np.sqrt(2),
    lfm_hdf4_path=lfm_hdf4_path,
    time=datetime(2015, 3, 17, 4, 45))

In [ ]:
Lstars = lstar_vs_nlocaltimes_experiment(mesh)

In [ ]:
plot_experiment(Lstars, 'T96')